In [1]:
import requests
import logging.config
import configparser
import warnings
from datetime import datetime, timedelta

In [2]:
def get_logger():
    """
    Initializes a logger object based on specified configurations.

    Returns:
    logging.Logger: A logger object.
    """
    logging.config.fileConfig('../conf/logging.conf')
    logger = logging.getLogger('exchangerates')
    return logger

In [3]:
def load_config(path):
    """
    Loads application configuration file.

    Parameters:
    path (str): Path to the configuration file.

    Returns:
    configparser.ConfigParser: An object of type ConfigParser.
    """
    config = configparser.ConfigParser()
    config.read(path)
    return config

In [4]:
def get_data(url):
    """
    Fetches data from a specified endpoint into json format.

    Parameters:
    url (str): The URL endpoint to fetch data from.

    Returns:
    dict: The JSON data fetched from the endpoint.
    """
    # Suppress InsecureRequestWarning
    warnings.filterwarnings("ignore", category=requests.packages.urllib3.exceptions.InsecureRequestWarning)

    response = requests.get(url, timeout=15, verify=True)
    response.raise_for_status()
    return response.json()

In [5]:
def get_rates_dataframe(data):
    """
    Converts a nested dictionary of exchange rates into a pandas DataFrame.

    Parameters:
    data (dict): A dictionary containing exchange rate data in a nested format.

    Returns:
    pd.DataFrame: A DataFrame containing exchange rate data with columns for Date, Currency, and Rate.
    """
    flat_rates = [(date, currency, rate) for date, currencies in data['rates'].items() for currency, rate in currencies.items()]
    df = pd.DataFrame(flat_rates, columns=['Date', 'Currency', 'Rate'])
    return df

In [6]:
def get_start_date(date_threshold):
    """
    Calculates a date based on the date threshold value.

    Parameters:
    date_threshold (int): Number of days to deduct from the current date.

    Returns:
    datetime.date: A date object.
    """
    return datetime.now().date() - timedelta(days=date_threshold)

In [7]:
def build_endpoint(config):
    """
    Builds an endpoint based on values in the configuration file.

    Parameters:
    config (ConfigParser): Configuration object.

    Returns:
    str: A complete endpoint.
    """
    section = dict(config.items('arguments'))

    url = section['endpoint']
    access_key = section['access_key']
    start_date = get_start_date(int(section['date_threshold']))
    end_date = datetime.now().date()
    base_currency = section['base_currency']
    symbols = section['symbols']

    return f"{url}?access_key={access_key}&start_date={start_date}&end_date={end_date}&base={base_currency}&symbols={symbols}"